# Outline of the full backend pipeline v0

In [8]:
import os
from transformers import AutoProcessor, BarkModel
from moviepy.editor import AudioClip, AudioFileClip, concatenate_audioclips
import scipy

import sys
sys.path.append('../audio_concatenation_v2')
from audio_functions_v2 import *

In [11]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'YOUR_TOKEN_HERE'

In [3]:
## Step 0.0: Load env
## Install using `pip install python-dotenv`

import dotenv
ENV_PATH = '../.env'
dotenv.load_dotenv(ENV_PATH)

True

# Step 0: Build vector DB

In [13]:
## `pip install pinecone-client`

from pinecone import Pinecone

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index = pc.Index("hackathon24s")

# Step 1: Get prompt from user in UI

In [26]:
USER_PROMPT = "Make me a podcast about LLMs"



# Step 2: Retrieve relevant clips from Vector DB

In [29]:
# install: `pip install -U sentence-transformers`

## Step 2.1: Embed the user prompt
from sentence_transformers import SentenceTransformer, util

embeddings_model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [43]:
query_embedding = embeddings_model.encode(USER_PROMPT)
test_embeddings = embeddings_model.encode(["make me a podcast about artificial intelligence", "make me a podcast about cooking"]) 

In [32]:
print("Similarity:", util.dot_score(query_embedding, test_embeddings))

Similarity: tensor([[133.5490, 129.7140]])


In [51]:
## Step 2.2: Query the index
NUM_RESULTS = 5

pinecone_results = index.query(vector=[query_embedding], top_k=NUM_RESULTS)
print(pinecone_results)

{'matches': [{'id': 'test', 'score': 1.0, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}


## Step 2A: Get transcript documents from SQL DB 

In [ ]:
# Returns text documents 
docs = retriever.get_relevant_documents(USER_PROMPT)

## Step 2B: Get audio clips from text documents

In [ ]:
# TODO: create a function that returns the file path to the audio clip (either from every doc individually or from an array of all docs)
def get_audio_clip_from_docs(docs):
    pass
# OR 
def get_audio_clip_from_doc(doc):
    pass

clips = get_audio_clip_from_docs(docs) or [get_audio_clip_from_doc(doc) for doc in docs]

# Step 3: Make video

## Step 3A: Instantiate model

In [9]:
# Instantiate model
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

voice_preset = "v2/en_speaker_6"

## Step 3B: Generate intro/outro clips

In [12]:
# Make intro sequence 
make_whole_show_intro(USER_PROMPT, model, processor, voice_preset)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
# Make outro clip
make_outro_clip(USER_PROMPT, model, processor, voice_preset)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Step 3C: Combine into final podcast

In [ ]:
# Combine all clips
all_clips = ['podlight_intro_clip.wav'] + clips + ['podlight_outro_clip.wav'] 
concatenate_clips(all_clips, 'full_podlight.wav')
# all done!